# Project: Classifying buildings post-Hurricane
Univ.AI <br>
AI-2 Cohort-4

## [DATASET](https://drive.google.com/file/d/1EH3p84xKMs_2m4ISSR7aHOQz61yxIv8L/view?usp=sharing)

### Names of the people who worked on the project:

Link to presentation: <br>
Link to slides: <br>
Link to poster: <br>
Link to notebook: <br>